In [ ]:
import gc
import os
import math
import pickle

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from operator import itemgetter

from sklearn.utils import shuffle
from collections import defaultdict

# from metric import PrintMetric

import warnings
warnings.filterwarnings("ignore")

raw_data_path = 'D:/git/Pigitt/vehicle_rec_sys/off_data'
new_data_path = 'D:/git/Pigitt/vehicle_rec_sys/recall_data'

os.makedirs(new_data_path, exist_ok=True)

In [ ]:
user_info = pd.read_csv(raw_data_path + '/user_info.txt', sep='\t') #, index_col=0
user_info.columns = ["user_id", "设备名称", "操作系统", "所在省", "所在市", "年龄","性别"]

user_info.head()

In [ ]:
doc_info = pd.read_table(raw_data_path + '/doc_info.txt', sep='\t')
doc_info.columns = ["item_id", "标题", "发文时间", "图片数量",  "一级分类", "二级分类", "关键词"]

doc_info.head()

In [ ]:
all_data = pd.read_csv(raw_data_path + '/train_data_30w.txt', sep='\t')#, index_col=0
all_data.columns = ["user_id", "item_id", "展现时间", "网路环境", "刷新次数", "展现位置", "是否点击", "消费时长（秒）"]

# all_data.head()

In [ ]:
all_data['展现时间'] = all_data['展现时间'].astype('str')
all_data['展现时间'] = all_data['展现时间'].apply(lambda x: int(x[:10]))

all_data['展现时间'] = pd.to_datetime(all_data['展现时间'], unit='s', errors='coerce')
all_data['展现时间_日期'] = all_data['展现时间'].dt.day

# all_data.head()

In [ ]:
mode = 'debug'
if mode == 'debug':
    all_data = shuffle(all_data)
    all_data.reset_index(drop=True)

    train_data = all_data[(all_data['展现时间_日期'] >= 5) & (all_data['展现时间_日期'] < 6)]
    test_data = all_data.loc[all_data['展现时间_日期'] == 6, :]
else:
    train_data = all_data[(all_data['展现时间_日期'] >= 1) & (all_data['展现时间_日期'] < 6)]
    test_data = all_data.loc[all_data['展现时间_日期'] == 6, :]

del all_data, doc_info, user_info
gc.collect()

In [ ]:
print(train_data['user_id'].nunique())
print(train_data['item_id'].nunique())

In [ ]:
item2users = train_data.groupby('item_id')['user_id'].apply(list).reset_index()

In [ ]:
item2users.head(1)

In [ ]:
user_set = set()
item_set = set()
user_sim_matrix = dict()
user_interacted_num = defaultdict(int)
item_interacted_num = defaultdict(int)  

In [ ]:
pbar = tqdm(total=item2users.shape[0])
for idx, row in item2users.iterrows():
    item_set.add(row['item_id'])
    user_set.update(row['user_id'])
    item_interacted_num[row['item_id']] += len(row['user_id'])
    for idx1, user_1 in enumerate(row['user_id']):
        user_interacted_num[user_1] += 1
        user_sim_matrix.setdefault(user_1, {})
        for idx2, user_2 in enumerate(row['user_id']):
            if user_1 == user_2:
                continue
            user_sim_matrix[user_1].setdefault(user_2, 0)
            # 热门物品用在计算用户之间相似度时，贡献小于非热门物品
            user_sim_matrix[user_1][user_2] += 1 / math.log(1+len(row['user_id']))
    pbar.update(1)
pbar.close()

In [ ]:
for user_1, related_users in tqdm(user_sim_matrix.items()):
    for user_2, weight in related_users.items():
        # 打压活跃用户
        user_sim_matrix[user_1][user_2] =\
            weight / math.sqrt(user_interacted_num[user_1] * user_interacted_num[user_2])

In [ ]:
user2items = train_data.groupby('user_id')['item_id'].apply(list)
popular_items = [val[0] for val in sorted(item_interacted_num.items(), key=lambda x: x[1], reverse=True)[:20]]

In [ ]:
user2items.head(2)

In [ ]:
user_rec = {}
rank = defaultdict(int)

In [ ]:

# 老用户
for relate_user, user_smi_score in sorted(user_sim_matrix[104028762].items(),key=itemgetter(1), reverse=True)[:50]:
    print(relate_user,user_smi_score)
    for candidate_item in user2items.loc[relate_user]:
        if candidate_item in user2items.loc[104028762]:
            continue
        rank[candidate_item] += user_smi_score

In [66]:
rec_items = [item[0] for item in sorted(rank.items(), key=itemgetter(1), reverse=True)[:20]]
# 如果推荐的物品不够，用热门物品进行填充
rec_items += popular_items[:20-len(rec_items)]
user_rec[104028762] = rec_items

In [ ]:
class UserCF(object):
    def __init__(self, his_data):
        self.user_set = set()
        self.item_set = set()

        self.his_data = his_data
        self.user_sim_matrix = dict()
        self.user_interacted_num = defaultdict(int)
        self.item_interacted_num = defaultdict(int)     # 热门推荐时会用到

    def calculate_similarity_matrix(self):
        item2users = self.his_data.groupby('item_id')['user_id'].apply(list).reset_index()

        # print(f'计算ItemCF第一阶段...')
        pbar = tqdm(total=item2users.shape[0])
        for idx, row in item2users.iterrows():
            self.item_set.add(row['item_id'])
            self.user_set.update(row['user_id'])
            self.item_interacted_num[row['item_id']] += len(row['user_id'])
            for idx1, user_1 in enumerate(row['user_id']):
                self.user_interacted_num[user_1] += 1
                self.user_sim_matrix.setdefault(user_1, {})
                for idx2, user_2 in enumerate(row['user_id']):
                    if user_1 == user_2:
                        continue
                    self.user_sim_matrix[user_1].setdefault(user_2, 0)
                    # 热门物品用在计算用户之间相似度时，贡献小于非热门物品
                    self.user_sim_matrix[user_1][user_2] += 1 / math.log(1 + len(row['user_id']))
            pbar.update(1)
        pbar.close()
        # 理论上，用户之间共现的物品越多，相似度越高
        # 但是，活跃用户与很多用户之间的相似度都很高
        # print(f'计算UserCF第二阶段...')
        for user_1, related_users in tqdm(self.user_sim_matrix.items()):
            for user_2, weight in related_users.items():
                # 打压活跃用户
                self.user_sim_matrix[user_1][user_2] =\
                    weight / math.sqrt(self.user_interacted_num[user_1] * self.user_interacted_num[user_2])

    def __call__(self, users, _n=50, _topk=20):
        print(f'开始ItemCF召回: Recall@{_topk}-Near@{_n}')
        user2items = self.his_data.groupby('user_id')['item_id'].apply(list)
        popular_items = [val[0] for val in sorted(
            self.item_interacted_num.items(), key=lambda x: x[1], reverse=True)[:_topk]]

        user_rec = {}
        for user_id in tqdm(users):
            # 新用户，直接推荐热门物品
            if user_id not in self.user_set:
                user_rec[user_id] = popular_items
            else:
                rank = defaultdict(int)
                for relate_user, user_smi_score in sorted(self.user_sim_matrix[user_id].items(),
                                                      key=itemgetter(1), reverse=True)[:_n]:
                    for candidate_item in user2items.loc[relate_user]:
                        # if candidate_item in user2items.loc[user_id]:
                        #     continue
                        rank[candidate_item] += user_smi_score
                rec_items = [item[0] for item in sorted(rank.items(), key=itemgetter(1), reverse=True)[:_topk]]
                # 如果推荐的物品不够，用热门物品进行填充
                rec_items += popular_items[:_topk-len(rec_items)]
                user_rec[user_id] = rec_items

        return user_rec

In [ ]:
ucf_cls_path = os.path.join(new_data_path, 'user_cf')
os.makedirs(ucf_cls_path, exist_ok=True)

In [ ]:
demo_ucf_path = os.path.join(ucf_cls_path, mode+'_ufc.pkl')

if os.path.exists(demo_ucf_path):
    with open(demo_ucf_path, 'rb') as file:
        demo_ucf = pickle.loads(file.read())
        file.close()
else:
    demo_ucf = UserCF(train_data)
    demo_ucf.calculate_similarity_matrix()
    demo_ucf_pkl = pickle.dumps(demo_ucf)

    output_ucf = open(demo_ucf_path, 'wb')
    output_ucf.write(demo_ucf_pkl)
    output_ucf.close()

In [ ]:
n, topk = 50, 100

# 召回
test_users = test_data['user_id'].unique()
icf_rec_result = demo_ucf(test_users, n, topk)

test_user_group = test_data.groupby('user_id')['item_id'].agg(list).reset_index()
test_pred = [icf_rec_result[user_id] for user_id in test_user_group['user_id']]
test_true = test_user_group['item_id'].to_list()

In [ ]:
PrintMetric(test_true, test_pred, topk)